In [1]:
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever

In [2]:
#fetch oai_key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("oai_key")

In [3]:
#load data
loader = PyPDFLoader("../data/MSFT_2022_Annual_Report.pdf")
pages = loader.load_and_split()
#index data into vector store
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())


In [4]:
llm = OpenAI(temperature=0.9, openai_api_key=os.environ["OPENAI_API_KEY"])

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [5]:

retriever = faiss_index.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [10]:

res = retrieval_chain.invoke({"input": "What is the summary of Fiscal Year 2022 Compared with Fiscal Year 2021, specially regarding SEGMENT RESULTS OF OPERATIONS figures?"})

In [12]:
print(res["answer"])



In Fiscal Year 2022, Microsoft saw an overall increase in revenue of $30.2 billion, driven by growth in all segments. This includes an increase of 18% in Productivity and Business Processes revenue, 25% in Intelligent Cloud revenue, and 10% in More Personal Computing revenue. Gross margin also increased by $19.8 billion, with a slight decrease in gross margin percentage. Operating expenses increased by $6.3 billion, driven by investments in cloud engineering, LinkedIn, Gaming, and commercial sales. Overall, operating income increased by $13.5 billion, with an increase in all segments. Current year net income and diluted EPS were positively impacted by a net tax benefit related to the transfer of intangible properties, while prior year figures were positively impacted by a net tax benefit related to the India Supreme Court decision on withholding taxes. Gross margin and operating income also included an unfavorable foreign currency impact of 2%.
